In [4]:
import os
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import numpy as np

from tqdm import tqdm
from torch.utils.data import DataLoader

from utility import *
from unet import UNet
from dataloader import CustomImageDataset, data_to_images, images_to_data

# Demo of the trained Colorization Model

For now just colorizes the images of a video, and stores them as a new video.

## 1. Data Loading

using the pytorch dataloader and dataset implementation from DataLoad.py

In [5]:
# HYPERPARAMETERS
dataset_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS'
subdir = 'val'
res = '176p'
method = 'deepflow'
use_flow = True

In [10]:
dataset = CustomImageDataset(dataset_path, subdir, res, method, use_flow)

dataset_size = len(dataset)
print('Dataset size:', dataset_size)

img1_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/train/176p/bear/00000.jpg'
img2_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/train/176p/bear/00081.jpg'
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

psnr = cv2.PSNR(img1, img2)
print(psnr)

vid labels: ['aerobatics', 'car-race', 'carousel', 'cats-car', 'chamaleon', 'deer', 'giant-slalom', 'girl-dog', 'golf', 'guitar-violin', 'gym', 'helicopter', 'horsejump-stick', 'hoverboard', 'lock', 'man-bike', 'monkeys-trees', 'mtb-race', 'orchid', 'people-sunset', 'planes-crossing', 'rollercoaster', 'salsa', 'seasnake', 'skate-jump', 'slackline', 'subway', 'tandem', 'tennis-vest', 'tractor']
Dataset size: 2056
12.088855117483794


## 2. Model Loading

In [ ]:


print("Cuda available: ", torch.cuda.is_available())

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

print("Using device: ", device)

In [ ]:
#initialize model
if use_flow:
    n_input_channels = 6
else:
    n_input_channels = 4

model = UNet(n_input_channels=n_input_channels)
model.to(device)

# Hyperparameters
learning_rate = 1e-3
batch_size = 16
epochs = 300

# Initialize the loss function
loss_fn = nn.MSELoss()


# Data loader
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
model_path = '~/Documents/Colorization/Models/'
UNet_folder = 'UNet'
model_path = os.path.expanduser(model_path)
UNet_folder = os.path.join(model_path, UNet_folder)

In [ ]:
model_folder = 'UNet_Flow_'+str(epochs)+'_epochs'

filename = 'UNet_Flow_epoch_295.pth'

resume(model, os.path.join(model_folder, filename))

## 3. Video Rendering

In [ ]:
# HYPERPARAMETERS
dataset_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS'
subdir = 'train'
res = '480p'
video_folder = 'AVI_videos'
fps = 24

# Run the actual function to make the images of a complete folder to videos
# all_images_2_video(dataset_path, subdir, res, video_folder, fps)

In [ ]:
dataset_path = '~/Documents/Colorization/Datasets/'
dataset_name = 'DAVIS'
image_folder = 'JPEGImages'

dataset_path = os.path.expanduser(dataset_path)
dataset_path = os.path.join(dataset_path, dataset_name)
img_folder_path = os.path.join(dataset_path, image_folder)
nd_array_path = os.path.join(dataset_path, 'nd_arrays')
nd_array_path = os.path.join(nd_array_path, '176p_deepflow')

resolution = '176p'
label = 'gray'
img_folder_path = os.path.join(img_folder_path, resolution+'_'+label)

save_label = 'UNet_Flow'

In [ ]:
for video_name in os.listdir(img_folder_path):
    print('Video name:', video_name)
    video_path = os.path.join(img_folder_path, video_name)
    save_path = '~/Documents/Colorization/Results/'
    save_path = os.path.expanduser(save_path)
    save_path = os.path.join(save_path, save_label, video_name)
    make_dir(save_path)
    still_path = os.path.join(save_path, 'stills')
    make_dir(still_path)

    n_frames = len(os.listdir(video_path))
    print('Number of frames:', n_frames)

    col_img = load_image(os.path.join(dataset_path, image_folder,resolution,video_name, '00000.jpg'))
    print(os.path.join(dataset_path, image_folder,resolution,video_name, '00000.jpg'))


    gray_img = None
    flow_img = None
    i = 1
    while i < n_frames:
        img_path = os.path.join(video_path, str(i).zfill(5)+'.jpg')
        gray_img = load_image(img_path)
        if use_flow:
            flow_frame = flow_frame = os.path.join(nd_array_path,video_name, str(i).zfill(5) + '.jpg.npy')
            flow_img = np.load(flow_frame)
        
        input, _ = images_to_data(col_img, flow_img, gray_img)
        
        prediction = model(input.unsqueeze(0).to(device))
        prediction = prediction.cpu()
        prediction = prediction.detach()
        prediction = prediction.squeeze(0)

        _, flow_img, grey_img, pred_img = data_to_images(input, prediction, use_flow=use_flow, input_only=False)

        col_img = cv2.cvtColor(pred_img, cv2.COLOR_RGB2BGR)

        if i%2==0:
            save_image(pred_img, os.path.join(still_path, str(i).zfill(5)+'.jpg'))
        else:
            save_image(cv2.cvtColor(pred_img, cv2.COLOR_RGB2BGR), os.path.join(still_path, str(i).zfill(5)+'.jpg'))

        col_img = pred_img
        i+=1

    images_2_video(still_path, os.path.join(save_path, video_name+'.avi'), fps=10)